<a href="https://colab.research.google.com/github/showmik121/Genetivie_AI/blob/main/Transformer_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

# 📌 ১. ডেটাসেট লোড করা
url = "https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv"
df = pd.read_csv(url)

# 📌 ২. অপ্রয়োজনীয় কলাম বাদ দিয়ে ফিচার এবং লেবেল আলাদা করা
df = df.drop(columns=['id', 'Unnamed: 32'])  # 'id' এবং আননেসেসারি কলাম বাদ

# টার্গেট লেবেল 'M' হলে 1, না হলে 0 (Malignant = 1, Benign = 0)
df['diagnosis'] = df['diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

# 📌 ৩. ফিচার এবং লেবেল তৈরি করা
X = df.drop(columns=['diagnosis']).values
y = df['diagnosis'].values

# 📌 ৪. ডেটা ট্রেন-টেস্ট ভাগ করা
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 ৫. ফিচার স্কেলিং (Normalization)
X_train = X_train / np.max(X_train)
X_test = X_test / np.max(X_test)

# 📌 ৬. Torch টেন্সরে রূপান্তর করা
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 📌 ৭. কাস্টম ডেটাসেট ক্লাস তৈরি করা
class CancerDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# 📌 ৮. DataLoader তৈরি করা
batch_size = 16
train_dataset = CancerDataset(X_train_tensor, y_train_tensor)
test_dataset = CancerDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 📌 ৯. ট্রান্সফর্মার ভিত্তিক নিউরাল নেটওয়ার্ক মডেল তৈরি করা
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads=2, hidden_dim=64, num_layers=1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)  # ইনপুট ফিচার থেকে হিডেন লেয়ার
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer_encoder(x.unsqueeze(1))  # Transformer encoder expects 3D input
        x = self.fc(x.squeeze(1))
        return self.sigmoid(x)

# 📌 ১০. মডেল ইনস্ট্যান্সিয়েট করা
input_dim = X_train.shape[1]
model = TransformerModel(input_dim)

# 📌 ১১. লস ফাংশন ও অপটিমাইজার সেট করা
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # L2 Regularization

# 📌 ১২. মডেল ট্রেন করা
num_epochs = 20
for epoch in range(num_epochs):
    total_loss = 0
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs.squeeze(1), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

# 📌 ১৩. মডেল টেস্ট করা
model.eval()  # Eval mode on
correct = 0
total = 0
with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features).squeeze(1)
        predicted = (outputs >= 0.5).float()  # Threshold 0.5 for classification
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")
